In [ ]:
!pip install pytorch-lightning

In [ ]:
!pip install py-rouge

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import Data_processing as dpros
import Emotion_model as EM
import test_functions as test_func
import statistics
from tqdm import tqdm
from torch.autograd import Variable
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl
import nltk
import random
import numpy as np
import pandas as pd
import torch.utils.data as utils
from torch.utils.data import Dataset, DataLoader
import nltk.translate.bleu_score as bleu

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Automatic Dialogue Generation with Expressed Emotions

## Data processing

In [ ]:

FILENAME = 'chat_emotion.txt'

lines = dpros.read_lines(filename=FILENAME)

# make every character lower case
lines = [ line.lower() for line in lines ]

lines = [ dpros.filter_line(line, dpros.EN_WHITELIST) for line in lines ]

qlines, asentence = dpros.filter_data(lines)

alines = []
tag = []

# separate the emotion from the sentence
for sentence in asentence:
    alines.append(sentence[:-2])
    tag.append(sentence[-1])

qtokenized = [ wordlist.split(' ') for wordlist in qlines ]
atokenized = [ wordlist.split(' ') for wordlist in alines ]

idx2w, w2idx = dpros.index_(qtokenized + atokenized, vocab_size=dpros.VOCAB_SIZE)

idx_q, idx_a, q_length = dpros.zero_pad(qtokenized, atokenized, w2idx)

metadata = {'w2idx': w2idx, 'idx2w': idx2w, 'limit': dpros.limit}


In [ ]:

word2id = metadata['w2idx']
id2word = metadata['idx2w']


## Model 1: Enc-bef

In [ ]:

class Final_model_light(pl.LightningModule):
    
    def __init__(self, emb_dim, enc_hid_dim, hid_dim, vocab_size, num_directions, pad_len, encoder, decoder, dropout):
        
        super(Final_model_light, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder

        self.embeddings = nn.Embedding(vocab_size, emb_dim)

        self.encoder2decoder = nn.Linear(enc_hid_dim * num_directions, hid_dim)
        self.tanh = nn.Tanh()
        self.out = nn.Linear(hid_dim, vocab_size)

    ## the forward method that makes the operations of the emotion embedding model and outputs the predictions
        
    def forward(self, input_sentence, input_length, target):
    
        input_sentence = self.embeddings(input_sentence)
        target = self.embeddings(target)

        src, (src_hidden, src_cell) = self.encoder(input_sentence, input_length)

        decoder_init_state = self.encoder2decoder(src_hidden)

        decoder_init_state = self.tanh(decoder_init_state)

        context = src.transpose(0, 1)

        trg, (hidden_trg, cell_trg) = self.decoder(target, (decoder_init_state, src_cell), context)

        logits = self.out(trg)

        return logits
    
    ## updates the embedding weights with the pre-trained weights:

    def load_word_embedding(self, id2word):

        embeddings_index = {}
        f = open('drive/My Drive/Colab Notebooks/cc.en.300.vec', 'r', encoding='utf-8')
        for line in tqdm(f):
            values = line.rstrip().rsplit(' ')
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        f.close()

        embedding_matrix = np.zeros((vocab_size, emb_dim))
        for i, word in id2word.items():
            if i < vocab_size:
                embedding_vector = embeddings_index.get(word)
                if embedding_vector is not None:
                    embedding_matrix[i] = embedding_vector
                else:
                    if word == '<pad>':
                        embedding_matrix[i] = np.zeros([emb_dim])
                    else:
                        embedding_matrix[i] = np.random.uniform(-1, 1, emb_dim)
        self.embeddings.weight = nn.Parameter(torch.FloatTensor(embedding_matrix))
        self.embeddings.requires_grad = False

    ## definition of the training step:
    
    def training_step(self, batch, batch_idx):
        
        src, src_length, trg = batch

        trg_1 = trg[:, :-1]
        trg_2 = trg[:, 1:]

        # makes the forward pass

        decoder_logit = self.forward(src, src_length, trg_1)

        ## calculate the loss

        loss = criterion(decoder_logit.contiguous().view(-1, vocab_size), trg_2.contiguous().view(-1))

        return {'loss': loss}

    ## at the end of the training step, print the average loss

    def training_epoch_end(self, outputs):
        
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        print("Train loss: ", avg_loss)
        return {'training_loss': avg_loss}
    
    def validation_step(self, batch, batch_idx):
        
        src, src_length, trg = batch

        trg_1 = trg[:, :-1]
        trg_2 = trg[:, 1:]

        # makes the forward pass

        decoder_logit = self.forward(src, src_length, trg_1)

        ## calculate the loss

        val_loss = criterion(decoder_logit.contiguous().view(-1, vocab_size), trg_2.contiguous().view(-1))

        return {'val_loss': val_loss}

    ## at the end of the validation step, print the average loss
    
    def validation_epoch_end(self, outputs):
        
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        print("Avg loss: ", avg_loss)
        return {'val_loss': avg_loss}
    
    def test_step(self, batch, batch_idx):
        
        src, src_length, trg = batch

        trg_1 = trg[:, :-1]
        trg_2 = trg[:, 1:]

        # makes the forward pass

        decoder_logit = self.forward(src, src_length, trg_1)

        # calculate the loss

        test_loss = criterion(decoder_logit.contiguous().view(-1, vocab_size), trg_2.contiguous().view(-1))

        return {'test_loss': test_loss}

    ## at the end of the test step, print the average loss
    
    def test_epoch_end(self, outputs):
        
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        return {'test_loss': avg_loss}
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.0001)
    
    def train_dataloader(self):
        
        train_loader = DataLoader(training_set, batch_size=dpros.batch_size, shuffle=True, num_workers=3)
        
        return train_loader
    
    def val_dataloader(self):
        
        valid_loader = DataLoader(valid_set, batch_size=dpros.batch_size, num_workers=3)
        
        return valid_loader
    
    def test_dataloader(self):
        
        test_loader = DataLoader(test_set, batch_size=dpros.batch_size, num_workers=3)
        
        return test_loader
        

### Train model

In [ ]:

## divide the sentences in train, test and validation. Then, add the emotion tokens to the source sentences and finally create the dataset objects for the three sets:

test_index = int(0.05 * len(idx_q))
train_index = int(len(idx_q) - test_index)
valid_index = int(train_index - int(0.95 * train_index))
train_index = int(0.95 * train_index)

X_train = [idx_q[i] for i in range(0, train_index)]
y_train = [idx_a[i] for i in range(0, train_index)]
length_train = [q_length[i] for i in range(0, train_index)]
tag_train = [tag[i] for i in range(0, train_index)]

X_valid = [idx_q[i] for i in range(train_index + 1, train_index + valid_index)]
y_valid = [idx_a[i] for i in range(train_index + 1, train_index + valid_index)]
length_valid = [q_length[i] for i in range(train_index + 1, train_index + valid_index)]
tag_valid = [tag[i] for i in range(train_index + 1, train_index + valid_index)]

X_test = [idx_q[i] for i in range(train_index + valid_index + 1, len(idx_q))]
y_test = [idx_a[i] for i in range(train_index + valid_index + 1, len(idx_q))]
length_test = [q_length[i] for i in range(train_index + valid_index + 1, len(idx_q))]
tag_test = [tag[i] for i in range(train_index + valid_index + 1, len(idx_q))]


X_train_emotion, length_trains = dpros.emotion_pad(X_train, length_train, tag_train, word2id)
X_valid_emotion, length_valids = dpros.emotion_pad(X_valid, length_valid, tag_valid, word2id)
X_test_emotion, length_tests = dpros.emotion_pad(X_test, length_test, tag_test, word2id)

training_set = utils.TensorDataset(torch.LongTensor(X_train_emotion), torch.LongTensor(length_trains), torch.LongTensor(y_train))
valid_set = utils.TensorDataset(torch.LongTensor(X_valid_emotion), torch.LongTensor(length_valids), torch.LongTensor(y_valid))
test_set = utils.TensorDataset(torch.LongTensor(X_test_emotion), torch.LongTensor(length_tests), torch.LongTensor(y_test))



In [ ]:

input_dim = len(word2id)
pad_len = 30
emb_dim = 300
dec_hid_dim = 600
enc_hid_dim = 300
hid_dim = 600
dropout = 0.75
vocab_size = len(word2id)
target_pad = word2id['<pad>']
num_directions = 2

encoder = EM.Encoder(input_dim, emb_dim, enc_hid_dim, dropout)
decoder = EM.Decoder(input_dim, emb_dim, dec_hid_dim, dropout)


In [ ]:

criterion = nn.CrossEntropyLoss().cuda()


In [ ]:

light_model = Final_model_light(emb_dim, enc_hid_dim, hid_dim, vocab_size, num_directions, pad_len, encoder, decoder, dropout)
light_model.load_word_embedding(id2word)

checkpoint_callback = ModelCheckpoint(filepath='drive/My Drive/Colab Notebooks/Model_checkpoint/', save_top_k=1, verbose=True, monitor='val_loss',mode='min')

trainer = pl.Trainer(gpus=1, max_epochs=3, log_save_interval=100000, weights_summary=None, log_gpu_memory=None, progress_bar_refresh_rate=0, default_root_dir='drive/My Drive/Colab Notebooks/Model_checkpoint/', checkpoint_callback=checkpoint_callback)

## trainer = pl.Trainer(resume_from_checkpoint='drive/My Drive/Colab Notebooks/Model_checkpoint/epoch=19.ckpt', gpus=1, max_epochs=20, log_save_interval=100000, weights_summary=None, log_gpu_memory=None, progress_bar_refresh_rate=0, default_root_dir='drive/My Drive/Colab Notebooks/Model_checkpoint/', checkpoint_callback=checkpoint_callback)


In [ ]:

trainer.fit(light_model)
trainer.test(light_model)


## Evaluation metrics

In [ ]:

test_loader = DataLoader(test_set, batch_size=dpros.batch_size, num_workers=3, drop_last=True)

answers, predict = test_func.create_answers_preds(light_model, test_loader, word2id, id2word)

print("Created successfully!")


Created successfully!


In [ ]:

for i in range(0, len(answers)):

    answers[i] = answers[i].split('<pad>')[0]
    predict[i] = predict[i].split('<pad>')[0]


### BLEU

In [ ]:

test_func.bleu_scores(answers, predict)


### ROUGE

In [ ]:

test_func.rouge_scores(answers, predict)


### Embedding average metric

In [ ]:

test_func.word_embedding_scores(answers, predict, light_model, word2id)


In [ ]:

## this is for creating the predictions for each emotion and store them in separate files

for tag in range(9):
    X_test_emotion, length_tests = dpros.emotion_pad(X_test, length_test, tag, word2id)
    test_set = utils.TensorDataset(torch.LongTensor(X_test_emotion), torch.LongTensor(length_tests), torch.LongTensor(y_test))
    test_loader = DataLoader(test_set, batch_size=dpros.batch_size, num_workers=3, drop_last=True)
    sources, predict = test_func.create_sources_preds(light_model, test_loader, word2id, id2word)

    print("Emotion " + str(tag))

    df = pd.DataFrame({'Sources': [''.join(source_test) for source_test in sources], 'Predictions': [''.join(predicted_test) for predicted_test in predict]})
    df.to_csv('Emotion' + str(tag) + '.csv', encoding='utf-8', index=False)
